In [1]:
!pip install sentence-transformers --no-index --find-links /kaggle/input/sentence-transformers/

Looking in links: /kaggle/input/sentence-transformers/
Processing /kaggle/input/sentence-transformers/sentence_transformers-2.7.0-py3-none-any.whl


In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from sklearn.metrics import cohen_kappa_score
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import keras_nlp
import pickle
import keras
import time
import gc
import torch
import tqdm
import numpy as np
import os
os.environ["KERAS_BACKEND"] = "jax" 
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="False"

2024-06-30 10:24:14.920985: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-30 10:24:14.921116: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-30 10:24:15.087711: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
BASE_PATH = os.getcwd()

## 1.Exploratory Data Analysis

## 1.1 Read the Data

In [4]:
#read the data 

df_train = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv")
print(f"We have about {df_train.shape[0]:,} essays")
df_test = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv")
df_train.head()

We have about 17,307 essays


,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4
3,001bdc0,"We all heard about Venus, the planet without a...",4
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3


## 1.2 Devide the data

In [5]:
#create our test set
#train_df, valid_df = train_test_split(df_train, test_size=0.2, stratify=df_train["score"],random_state=42)

In [6]:
#train_df.head()

## 1.3 Explore the data

What's the score distribution across the essays?

In [7]:
#visualize the score distribution across the essays
#sns.countplot(data=train_df,x="score",order=train_df["score"].value_counts().index)

Is there a correlation between score and essay length or number of words?

There seems to be **no** relationship between the length of the essay and the score the student gets.

What's the distribution of tokens ?

In [8]:
#load the tokenizer
#tokenizer = keras_nlp.models.DebertaV3Tokenizer.from_preset("deberta_v3_base_en")


In [9]:
#compute token length of each essay across the dataset
#train_df["number_of_tokens"] = train_df["full_text"].apply(lambda x:tokenizer(x).shape[0])

#define a figure object of custom dimensions
#plt.figure(figsize=(8,8),dpi=200)

#plot the histogram
#sns.displot(data=train_df,x="number_of_tokens",kind="hist",kde=True)

In [10]:
#train_df["number_of_tokens"].describe()

In [11]:
#compute the number of words per essays
#train_df["essay_length"] = train_df["full_text"].apply(lambda x:len(x.split(" ")))

#plot the scatter plot

#sns.scatterplot(data=train_df,x="score",y="essay_length")

# 2.Prepare the data

## Define some useful functions

## Generate Embedding of the texts

In [12]:
def embed(arr,model,num_of_gpus=torch.cuda.device_count(),batch_size=5):
    if num_of_gpus >1:
        #devices = [f"cuda:{i}" for i in range(num_of_gpus)]
        pool = model.start_multi_process_pool()
        print("start embedding...")
        t1 = time.time()
        embeddings = model.encode_multi_process(arr,
                                                pool=pool,
                                                batch_size=batch_size
                                               )
        print(f"Finished... took {(time.time() - t1) /60} min")
    else:
    
            print("start embedding...")
            embeddings = model.encode(arr,
                                batch_size=batch_size,device="cuda",
                                show_progress_bar=True)
            t1 = time.time()
            print(f"Finished... took {(time.time() - t1) /60} min")

    return embeddings
    

In [13]:
embedding_models_list = [#'llmrails',
                         'alibaba',
                        "mpnet",
                        #'uae',
                        #'baai',#'intfloat',
                        #"all-mpnet-base-v2",
                        #'Alibaba-NLP/gte-large-en-v1.5'
]
basis = "/kaggle/input/source-file-nb2/"
#basis = ""


def generate_embeddings(df=df_train,embedding_models_list=embedding_models_list,num_of_gpus=torch.cuda.device_count()):
    t1 = time.time()
    arr = df["full_text"].to_numpy()
    model = SentenceTransformer(basis+embedding_models_list[0], trust_remote_code=True)
    print(f"Embedding using {embedding_models_list[0]} ")
    final_embedding = embed(arr,model,num_of_gpus)
    del model
    
     # Clear the cache
    torch.cuda.empty_cache()
    gc.collect()
    for model_name in tqdm.tqdm(embedding_models_list[1:]):
        print(f"Embedding using {model_name} ")
        try:
            model = SentenceTransformer(basis+model_name, trust_remote_code=True)
            embeddings = embed(arr,model,num_of_gpus)
            del model
            
            # Clear the cache
            torch.cuda.empty_cache()
            gc.collect()
            final_embedding = np.append(final_embedding,embeddings,axis=1)
        except:
            print("Execption") 
    print(f"Overall process Finished... took {(time.time() - t1) /60} min")
    return final_embedding

#final_embedding = generate_embeddings()

In [14]:
test_embeddings = generate_embeddings(df_test,num_of_gpus=1)



Embedding using alibaba 
start embedding...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Finished... took 1.9868214925130207e-08 min


  0%|          | 0/1 [00:00<?, ?it/s]

Embedding using mpnet 
start embedding...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Finished... took 1.5894571940104166e-08 min


100%|██████████| 1/1 [00:03<00:00,  3.61s/it]

Overall process Finished... took 0.2945047458012899 min


import pickle
with open("/kaggle/working/final_embedding.pkl","wb") as file:
    pickle.dump(final_embedding,file)

In [15]:
with open("/kaggle/input/good-embedding/final_embedding.pkl","rb") as file:
    final_embedding =  pickle.load(file)

In [16]:
#from numba import cuda 
#device = cuda.get_current_device()
#device.reset()

In [17]:
def to_ordinal(y, num_classes=None, dtype="float32"):
    """Converts a class vector (integers) to an ordinal regression matrix.

    This utility encodes class vector to ordinal regression/classification
    matrix where each sample is indicated by a row and rank of that sample is
    indicated by number of ones in that row.

    Args:
        y: Array-like with class values to be converted into a matrix
            (integers from 0 to `num_classes - 1`).
        num_classes: Total number of classes. If `None`, this would be inferred
            as `max(y) + 1`.
        dtype: The data type expected by the input. Default: `'float32'`.

    Returns:
        An ordinal regression matrix representation of the input as a NumPy
        array. The class axis is placed last.
    """
    y = np.array(y, dtype="int")
    input_shape = y.shape

    # Shrink the last dimension if the shape is (..., 1).
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])

    y = y.reshape(-1)
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    range_values = np.arange(num_classes - 1)
    range_values = np.tile(np.expand_dims(range_values, 0), [n, 1])
    ordinal = np.zeros((n, num_classes - 1), dtype=dtype)
    ordinal[range_values < np.expand_dims(y, -1)] = 1
    output_shape = input_shape + (num_classes - 1,)
    ordinal = np.reshape(ordinal, output_shape)
    return ordinal

In [18]:
#prepare the data
from keras.utils import to_categorical
#build the features and labels
X = final_embedding
#y = train_df["score"].to_numpy() 
y = df_train["score"].to_numpy()
#y = to_categorical(y)[:,1:]

print(f"Xtrain shape: {X.shape}")
print(f"ytrain shape: {y.shape}")

Xtrain shape: (17307, 1792)
ytrain shape: (17307,)


In [19]:
X,X_val,y,y_val = train_test_split(final_embedding,y, test_size=0.2,random_state=42,stratify=y)
y_val = to_ordinal(y_val)
y = to_ordinal(y)

In [20]:
import math
import matplotlib.pyplot as plt


def get_lr_callback(batch_size=100, mode='cos', epochs=10, plot=False):
    lr_start, lr_max, lr_min = 0.6e-5, 0.3e-5 * batch_size, 0.3e-5
    lr_ramp_ep, lr_sus_ep, lr_decay = 2, 0, 0.75

    def lrfn(epoch):  # Learning rate update function
        if epoch < lr_ramp_ep: lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
        elif epoch < lr_ramp_ep + lr_sus_ep: lr = lr_max
        elif mode == 'exp': lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
        elif mode == 'step': lr = lr_max * lr_decay**((epoch - lr_ramp_ep - lr_sus_ep) // 2)
        elif mode == 'cos':
            decay_total_epochs, decay_epoch_index = epochs - lr_ramp_ep - lr_sus_ep + 3, epoch - lr_ramp_ep - lr_sus_ep
            phase = math.pi * decay_epoch_index / decay_total_epochs
            lr = (lr_max - lr_min) * 0.5 * (1 + math.cos(phase)) + lr_min
        return lr

    if plot:  # Plot lr curve if plot is True
        plt.figure(figsize=(10, 5))
        plt.plot(np.arange(epochs), [lrfn(epoch) for epoch in np.arange(epochs)], marker='o')
        plt.xlabel('epoch'); plt.ylabel('lr')
        plt.title('LR Scheduler')
        plt.show()


    return keras.callbacks.LearningRateScheduler(lrfn, verbose=False)  # Create lr callback

lr_cb = get_lr_callback(10, plot=False)

ckpt_cb = keras.callbacks.ModelCheckpoint(
    "best_model.weights.h5",
    monitor="val_weighted_kappa",
    save_best_only=True,
    save_weights_only=True,
    mode="max",
)

#buuild the early stopping call back
early_callback = keras.callbacks.EarlyStopping(
    monitor='val_weighted_kappa',
    min_delta=0,
    patience=10,
    verbose=1,
    mode='max',
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=0
)

In [21]:
class WeightedKappa(keras.metrics.Metric):
    def __init__(self, num_classes=6, epsilon=1e-6):
        super().__init__(name="weighted_kappa")
        self.num_classes = num_classes
        self.epsilon = epsilon

        label_vec = keras.ops.arange(num_classes, dtype=keras.backend.floatx())
        self.row_label_vec = keras.ops.reshape(label_vec, [1, num_classes])
        self.col_label_vec = keras.ops.reshape(label_vec, [num_classes, 1])
        col_mat = keras.ops.tile(self.col_label_vec, [1, num_classes])
        row_mat = keras.ops.tile(self.row_label_vec, [num_classes, 1])
        self.weight_mat = (col_mat - row_mat) ** 2

        self.numerator = self.add_weight(name="numerator", initializer="zeros")
        self.denominator = self.add_weight(name="denominator", initializer="zeros")
        self.o_sum = self.add_weight(name = 'o_sum', initializer = 'zeros')
        self.e_sum = self.add_weight(name = 'e_sum', initializer = 'zeros')

    def update_state(self, y_true, y_pred, **args):
        # revert ordinal regression labels to classification labels
        p = keras.ops.sum(y_true, axis=-1) - 1
        p = keras.ops.cast(p,dtype="int8")
        y_true = keras.ops.one_hot(p, 6, dtype="int8")
        y_pred = keras.ops.one_hot(
            keras.ops.sum(keras.ops.cast(y_pred > 0.5, dtype="int8"), axis=-1) - 1, 6
        )
        # weighted kappa calculation
        y_true = keras.ops.cast(y_true, dtype=self.col_label_vec.dtype)
        y_pred = keras.ops.cast(y_pred, dtype=self.weight_mat.dtype)
        batch_size = keras.ops.shape(y_true)[0]

        cat_labels = keras.ops.matmul(y_true, self.col_label_vec)
        cat_label_mat = keras.ops.tile(cat_labels, [1, self.num_classes])
        row_label_mat = keras.ops.tile(self.row_label_vec, [batch_size, 1])

        weight = (cat_label_mat - row_label_mat) ** 2

        self.numerator.assign_add(keras.ops.sum(weight * y_pred))
        label_dist = keras.ops.sum(y_true, axis=0, keepdims=True)
        pred_dist = keras.ops.sum(y_pred, axis=0, keepdims=True)
        w_pred_dist = keras.ops.matmul(
            self.weight_mat, keras.ops.transpose(pred_dist, [1, 0])
        )
        self.denominator.assign_add(
            keras.ops.sum(keras.ops.matmul(label_dist, w_pred_dist))
        )
        
        self.o_sum.assign_add(keras.ops.sum(y_pred))
        self.e_sum.assign_add(keras.ops.sum(
            keras.ops.matmul(keras.ops.transpose(label_dist, [1, 0]), pred_dist)
        ))

    def result(self):
        return 1.0 - (
            keras.ops.divide_no_nan(self.numerator, self.denominator)
            * keras.ops.divide_no_nan(self.e_sum, self.o_sum)
        )

    def reset_state(self):
        self.numerator.assign(0)
        self.denominator.assign(0)
        self.o_sum.assign(0)
        self.e_sum.assign(0)

# 3.Training

In [22]:
import keras_tuner

In [23]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import BatchNormalization
#build the DNN
keras.utils.set_random_seed(101)
def build_model(hp):

    model = Sequential()
    for i in range(hp.Int("num_layers",1,10)):
        model.add(
            keras.layers.Dense(
            units=hp.Int(f"units_{i}",min_value=32,max_value=2096,step=32),
            activation="relu",
            )
        )
    if hp.Boolean("dropout"):
        model.add(keras.layers.Dropout(0.25))
    if hp.Boolean("bacthnormalization"):
        model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(6,activation="sigmoid"))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    #compile the model

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=[WeightedKappa()],
    )
    return model


#build the model

build_model(keras_tuner.HyperParameters())



<Sequential name=sequential, built=False>

In [24]:
tuner = keras_tuner.BayesianOptimization(
    hypermodel=build_model,
    objective=keras_tuner.Objective("val_weighted_kappa", direction="max"),
    max_trials=5,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)

In [25]:
tuner.search_space_summary()


Search space summary
Default search space size: 5
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 10, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 2096, 'step': 32, 'sampling': 'linear'}
dropout (Boolean)
{'default': False, 'conditions': []}
bacthnormalization (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [26]:
tuner.search(X, y, epochs=50, validation_data=(X_val, y_val))

Trial 5 Complete [00h 03m 27s]
val_weighted_kappa: 0.7590270042419434

Best val_weighted_kappa So Far: 0.7648723423480988
Total elapsed time: 00h 16m 57s


In [27]:
models = tuner.get_best_models(num_models=2)
best_model = models[0]
best_model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 46 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 288)            │       516,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1504)           │       434,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 320)            │       481,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1568)           │       503,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │       100,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1696)           │       110,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1248)           │     2,117,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 640)            │       799,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1920)           │     1,230,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1920)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1920)           │         7,680 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 6)              │        11,526 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,313,766 (24.09 MB)

 Trainable params: 6,309,926 (24.07 MB)

 Non-trainable params: 3,840 (15.00 KB)

In [28]:
x_all = np.concatenate((X, X_val))
y_all = np.concatenate((y, y_val))

In [29]:
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
model = build_model(best_hps[0])
# Fit with the entire dataset.

model.fit(x=x_all, y=y_all, epochs=20,callbacks=[early_callback])

Epoch 1/20
 31/541 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.5156 - weighted_kappa: -0.0092

W0000 00:00:1719744125.649460      83 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


541/541 ━━━━━━━━━━━━━━━━━━━━ 22s 29ms/step - loss: 0.3119 - weighted_kappa: 0.4078
Epoch 2/20
 31/541 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.2212 - weighted_kappa: 0.6223

W0000 00:00:1719744141.262307      85 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
/opt/conda/lib/python3.10/site-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_weighted_kappa` which is not available. Available metrics are: loss,weighted_kappa
  current = self.get_monitor_value(logs)


541/541 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.2105 - weighted_kappa: 0.6648
Epoch 3/20
541/541 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.1908 - weighted_kappa: 0.7057
Epoch 4/20
541/541 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.1816 - weighted_kappa: 0.7197
Epoch 5/20
541/541 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.1757 - weighted_kappa: 0.7343
Epoch 6/20
541/541 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.1725 - weighted_kappa: 0.7423
Epoch 7/20
541/541 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.1699 - weighted_kappa: 0.7474
Epoch 8/20
541/541 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.1667 - weighted_kappa: 0.7586
Epoch 9/20
541/541 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.1634 - weighted_kappa: 0.7648
Epoch 10/20
541/541 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.1615 - weighted_kappa: 0.7665
Epoch 11/20
541/541 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.1589 - weighted_kappa: 0.7731
Epoch 12/20
541/541 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.1578 - weighted_kappa: 0.7

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Test the Model on the validation Set

del X
del X_val
del y
del y_val
del final_embedding


# Clear the cache
torch.cuda.empty_cache()
gc.collect()

for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) and obj.is_cuda:
            del obj
    except Exception as e:
        pass

# Reset GPU memory stats
torch.cuda.reset_max_memory_allocated()
torch.cuda.reset_max_memory_cached()

from sklearn.metrics import cohen_kappa_score


#find the right thresold 
def find_thresholds(true, pred, steps=50):

    # SAVE TRIALS FOR PLOTTING
    xs = [[],[],[],[],[]]
    ys = [[],[],[],[],[]]

    # COMPUTE BASELINE METRIC
    threshold = [1.5, 2.5, 3.5, 4.5, 5.5]
    pred2 = pd.cut(pred, [-np.inf] + threshold + [np.inf], 
                    labels=[1,2,3,4,5,6]).astype('int32')
    best = cohen_kappa_score(true, pred2, weights="quadratic")

    # FIND FIVE OPTIMAL THRESHOLDS
    for k in range(5):
        for sign in [1,-1]:
            v = threshold[k]
            threshold2 = threshold.copy()
            stop = 0
            while stop<steps:

                # TRY NEW THRESHOLD
                v += sign * 0.001
                threshold2[k] = v
                pred2 = pd.cut(pred, [-np.inf] + threshold2 + [np.inf], 
                                labels=[1,2,3,4,5,6]).astype('int32')
                metric = cohen_kappa_score(true, pred2, weights="quadratic")

                # SAVE TRIALS FOR PLOTTING
                xs[k].append(v)
                ys[k].append(metric)

                # EARLY STOPPING
                if metric<=best:
                    stop += 1
                else:
                    stop = 0
                    best = metric
                    threshold = threshold2.copy()

    # COMPUTE FINAL METRIC
    pred2 = pd.cut(pred, [-np.inf] + threshold + [np.inf], 
                    labels=[1,2,3,4,5,6]).astype('int32')
    best = cohen_kappa_score(true, pred2, weights="quadratic")   

    # RETURN RESULTS
    threshold = [np.round(t,3) for t in threshold]
    return best, threshold, xs, ys

y_pred = lgbm_classifier.predict(X_val)
#y_true = valid_df["score"].to_numpy()
best, threshold, xs, ys = find_thresholds(y_val,y_pred.reshape(-1))

#print(f"Kappa score:{cohen_kappa_score(y_true,y_pred,weights='quadratic')}")
print(f"Kappa score:{best}")

def thresholding (output):
    if output < threshold[0]:
        return 1
    elif output >= threshold[0] and output < threshold[1]:
        return 2
    elif output >= threshold[1] and output < threshold[2]:
        return 3
    elif output >= threshold[2] and output < threshold[3]:
        return 4
    elif output >= threshold[3] and output < threshold[4]:
        return 5
    else:
        return 6


output = model.predict(test_embeddings)
print(output)
results = list(map(thresholding,output))

df_test["score"] = output
df_test

df_test[["essay_id","score"]].to_csv("submission.csv",index=False)

In [30]:
#test_embeddings = generate_embeddings(df_test,num_of_gpus=1)
test_preds = model.predict(test_embeddings, verbose=1)

# Convert probabilities to class labels
test_preds = np.sum((test_preds>0.5).astype(int), axis=-1).clip(1, 6)

# Create a DataFrame to store the submission
sub_df = df_test[["essay_id"]].copy()

# Add the formatted predictions to the submission DataFrame
sub_df["score"] = test_preds

# Save Submission
sub_df.to_csv('submission.csv',index=False)

# Display the first 2 rows of the submission DataFrame
sub_df.head()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 703ms/step


,essay_id,score
0,000d118,3
1,000fe60,3
2,001ab80,4


y_pred = model2.predict(preprocessor(valid_df["full_text"]))
y_true = valid_df["score"].to_numpy()
best, threshold, xs, ys = find_thresholds(y_true,y_pred.reshape(-1))

#print(f"Kappa score:{cohen_kappa_score(y_true,y_pred,weights='quadratic')}")
print(f"Kappa score:{best}")

def thresholding (output):
    if output < threshold[0]:
        return 1
    elif output >= threshold[0] and output < threshold[1]:
        return 2
    elif output >= threshold[1] and output < threshold[2]:
        return 3
    elif output >= threshold[2] and output < threshold[3]:
        return 4
    elif output >= threshold[3] and output < threshold[4]:
        return 5
    else:
        return 6

df_test = pd.read_csv(f"{BASE_PATH}{path}test.csv")
text = df_test["full_text"].to_list()

print(text)
processed_text = preprocessor(text)
print(processed_text)
output = model2.predict(processed_text)
print(output)
results = list(map(thresholding,output))
print(results)
